In [123]:
from threading import Lock
import warnings
import matplotlib.pyplot as plt
import pyximport
from tqdm import tnrange, tqdm_notebook

warnings.filterwarnings('ignore')

import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), os.pardir))
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

pyximport.install(setup_args={"include_dirs": np.get_include()})
import XXZZham as XXZZham
from XXZZham import add_high_energies, rotate_to_00_base
import random
import adiabatic_sim as asim
import time

import multiprocessing
import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(multiprocessing.cpu_count())))
import os
import LH_tools as LHT

%precision %.15f
PRECISION  = 2 ** -40

from IPython.display import display
import FD_tools as FDT
from numpy import angle,pi

In [18]:
def create_back_and_forward_props_return_EV(tlist, H_0, H_1):
    """

    Creates a back and forwared propagators
    :param tlist:
    :param H_0:
    :param H_1:
    :return:
    """
    EVs_forward = []
    n = len(H_0.dims[0])
    prop = tensor([qeye(2)] * n)
    tmax = max(tlist)
    last_time = 0
    for time in tlist:
        dt = abs(time - last_time)
        s = time / tmax
        Hs = H_0 * (1 - s) + H_1 * s
        EVs_forward.append(Hs.eigenstates(eigvals=2)[0])
        U = (-1j * Hs * dt).expm()
        prop = U * prop
        last_time = time
    last_time = 0
    backprop = tensor([qeye(2)] * n)
    for time in tlist[1:]:
        dt = abs(time - last_time)
        s = time / tmax
        Hs = H_0 * (s) + H_1 * (1 - s)
        U = (-1j * Hs * dt).expm()
        backprop = U * backprop
        last_time = time
    return backprop, prop,EVs_forward


In [126]:
def create_half_prop(tlist, H_0, H_1, epsilon =0.1):
    """

    Creates a back and forwared propagators
    :param tlist:
    :param H_0:
    :param H_1:
    :return:
    """
    EVs_forward = []
    n = len(H_0.dims[0])
    prop = tensor([qeye(2)] * n)
    tmax = max(tlist)
    last_time = 0
    for time in tlist:
        if time > tlist[-1]/2 - epsilon :
            return prop
        dt = abs(time - last_time)
        s = time / tmax
        Hs = H_0 * (1 - s) + H_1 * s
        EVs_forward.append(Hs.eigenstates(eigvals=2)[0])
        U = (-1j * Hs * dt).expm()
        prop = U * prop
        last_time = time


In [19]:
H0n, H1n, psi0n =  FDT.prepare_groverlike_system(15)
T12 = 4.84343434343
tlist = np.linspace(0,T12,150)
backprop, forwardprop,EVs_forward = create_back_and_forward_props_return_EV(tlist, H0n, H1n)

In [41]:

phase0 = sum([ev1 for ev1,ev2 in  (EVs_forward[len(EVs_forward)//2:])])
phase1 = sum([ev2 for ev1,ev2 in  (EVs_forward[len(EVs_forward)//2:])])

In [71]:
dt = tlist[1] - tlist[0]

In [106]:
psi = forwardprop * psi0n 
phi0_proj = LHT.proj_on(basis(2,0))
phi1_proj = LHT.proj_on(basis(2,1))
display(np.exp(1j*phase0*dt*2) * phi0_proj* psi)
display(np.angle((np.exp(1j*phase0*dt*2) * phi0_proj* psi)[0][0][0]))
display(np.exp(1j*phase1*dt*2) * phi1_proj* psi)
display(np.angle((np.exp(1j*phase1*dt*2)*phi1_proj* psi)[1][0][0]))

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.00679386+0.00634206j]
 [ 0.00000000+0.j        ]]

0.751017023498449

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.00000000+0.j        ]
 [ 0.33670231+0.94156528j]]

1.227368367842261

In [120]:
np.angle((np.exp(-1j*phase0*dt*2)*phi0_proj* psi)[0][0][0])

-1.670186314989131

In [121]:
np.angle((np.exp(-1j*(phase1+phase0)*dt)*phi1_proj* psi)[1][0][0])

1.407244422978723

In [124]:
pi/4

0.785398163397448

In [80]:
phase0*dt
phase1*dt

1.832669472473743

In [79]:
T12/8+ T12/4

1.816287878786250

In [146]:
half_prop12 = create_half_prop(tlist, H0n, H1n,0.01)

In [147]:
half_prop12*psi0n

Quantum object: dims = [[2], [1]], shape = [2, 1], type = ket
Qobj data =
[[ 0.00686103-0.00089976j]
 [ 0.82225480-0.56907747j]]

In [160]:
epsilon = 0.5
H_half_epsilon = H0n*(1/2+epsilon)+H1n*(1/2-0.1*epsilon)
ev_he0 ,ev_he1  = H_half_epsilon.eigenstates()[1]
(LHT.proj_on(ev_he0)+ LHT.proj_on(ev_he1)).data

array([ 1.+0.j,  1.+0.j])

In [161]:
H0n

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = True
Qobj data =
[[  9.99969482e-01  -5.50909700e-03]
 [ -5.50909700e-03   3.05175781e-05]]

In [162]:
H1n

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = True
Qobj data =
[[ 0.  0.]
 [ 0.  1.]]